# Dask Geohash Sorted

In [1]:
import logging
import time
from datetime import datetime
from pathlib import Path
from shapely.geometry import Polygon, box
from polygon_geohasher.polygon_geohasher import polygon_to_geohashes, geohashes_to_polygon
import geohash
from functools import reduce

import numpy as np
import pandas as pd
import geopandas as gpd
import dask.dataframe as dd
from distributed import LocalCluster, Client

import spatialpandas as spd
from spatialpandas.io import read_parquet, read_parquet_dask
import geopandas as gpd
from pathlib import Path
from distributed import LocalCluster, Client
import numpy as np
import dask.dataframe as dd

In [ ]:
cluster = LocalCluster(#silence_logs=logging.ERROR,
                       dashboard_address=':8790',
                       n_workers=4,
                       threads_per_worker=2,
                       memory_limit='3 GB')
client = Client(cluster)
client

In [3]:
base_path = Path('../../')
contiguous_us_bounding_box = box(-124.848974, 24.396308, -66.885444, 49.384358)

In [4]:
# load spatially sorted us data
spatial_sort_path = base_path / 'data/us_cont_spatiallysorted.parquet'
df = read_parquet_dask(spatial_sort_path)
df.head(2)

,position,latitude,longitude
hilbert_distance,,,
25629,"Point([-124.443, 24.447])",24.447,-124.443
99850,"Point([-124.532, 24.704])",24.704,-124.532


In [5]:
len_df = len(df)
len_df

113944489

In [11]:
%%time
# Save various size subsets of the zip code data

zips_1 = gpd.read_file(base_path / f'data/zip_codes/zips_1.geojson').loc[:, ['geometry']]
zips_1 = spd.geodataframe.GeoDataFrame(zips_1, geometry='geometry')
zips_10 = gpd.read_file(base_path / f'data/zip_codes/zips_10.geojson').loc[:, ['geometry']]
zips_10 = spd.geodataframe.GeoDataFrame(zips_10, geometry='geometry')
zips_100 = gpd.read_file(base_path / f'data/zip_codes/zips_100.geojson').loc[:, ['geometry']]
zips_100 = spd.geodataframe.GeoDataFrame(zips_100, geometry='geometry')
zips_1000 = gpd.read_file(base_path / f'data/zip_codes/zips_1000.geojson').loc[:, ['geometry']]
zips_1000 = spd.geodataframe.GeoDataFrame(zips_1000, geometry='geometry')
zips_10000 = gpd.read_file(base_path / f'data/zip_codes/zips_10000.geojson').loc[:, ['geometry']]
zips_10000 = spd.geodataframe.GeoDataFrame(zips_10000, geometry='geometry')

CPU times: user 43.5 s, sys: 952 ms, total: 44.5 s
Wall time: 44.1 s


In [12]:
%%time
total_points = len_df
num_partitions = df.npartitions
num_polygons = []
time_sec = []
num_result_points = []
num_points = len_df # len(df.partitions[:num_partitions])

t00 = time.time()
for zip_gdf in [zips_1, zips_10, zips_100, zips_1000, zips_10000]:
    num_polygons.append(len(zip_gdf))
    t0 = time.time()
    
    rdf = spd.sjoin(df, zip_gdf, how='inner').compute()

    time_sec.append(time.time() - t0)
    num_result_points.append(len(rdf))
    print(f'num_polygons[-1]: {num_polygons[-1]}, time_sec[-1]: {time_sec[-1]:.0f} s')

num_polygons[-1]: 1, time_sec[-1]: 1 s
num_polygons[-1]: 10, time_sec[-1]: 3 s
num_polygons[-1]: 100, time_sec[-1]: 10 s
num_polygons[-1]: 1000, time_sec[-1]: 21 s
num_polygons[-1]: 10000, time_sec[-1]: 57 s
CPU times: user 9.74 s, sys: 2.35 s, total: 12.1 s
Wall time: 1min 32s


In [15]:
results_df = pd.DataFrame({'num_polygons': num_polygons,
                           'num_points': num_points,
                           'num_result_points': num_result_points,
#                            'sort_time_sec': 0,
                           'time_min': np.asarray(time_sec)/60})
#                            'total_points': total_points})
# results_df['projected_total_time_hr'] = results_df.time_min*total_points/num_points/60                           
results_df.to_csv(f'spatially_sorted_results_{datetime.now()}.csv')
results_df

,num_polygons,num_points,num_result_points,time_min
0,1,113944489,1031,0.009435
1,10,113944489,6551,0.052584
2,100,113944489,203284,0.171888
3,1000,113944489,2403824,0.352971
4,10000,113944489,25877947,0.952060
